In [0]:
val datosDF = spark.read.option("inferSchema","true")
            .option("header","true")
            .option("delimiter","\t")
            .csv("/home/david/Datos_ENEMDU_PEA_v2.csv")


In [1]:
val data2 = datosDF.drop("id_persona")

In [2]:
datosDF.select("ingreso_laboral").where($"ingreso_laboral" >= 390).count


In [3]:
datosDF.select("sectorizacion").where($"sectorizacion".isNull).count

In [4]:
datosDF.select("nivel_de_instruccion").groupBy("nivel_de_instruccion").count().orderBy(desc("count")).show(false)

In [5]:
datosDF.select("rama_actividad").where($"ingreso_laboral" >= 1000).groupBy("rama_actividad").count.orderBy(desc("count")).show

In [6]:
val provincias=data2.withColumn("provincia", when($"provincia" ===  1 , lit("Azuay"))
                .otherwise(when($"provincia" ===  2 , lit("Bovilar"))
                        .otherwise(when($"provincia" === 3, lit("Caniar"))
                        .otherwise(when($"provincia" === 4, lit("Carchi"))
                        .otherwise(when($"provincia" === 5, lit("Cotopaxi"))
.otherwise(when($"provincia" === 6, lit("Chimborazo"))
.otherwise(when($"provincia" === 7, lit("El Oro"))
.otherwise(when($"provincia" === 8, lit("Esmeraldas"))
.otherwise(when($"provincia" === 9, lit("Guayas"))
.otherwise(when($"provincia" === 10, lit("Imbabura"))
.otherwise(when($"provincia" === 11, lit("Loja"))
.otherwise(when($"provincia" === 12, lit("Los Rios"))
.otherwise(when($"provincia" === 13, lit("Manabi"))
.otherwise(when($"provincia" === 14, lit("Morona Santiago"))
.otherwise(when($"provincia" === 15, lit("Napo"))
.otherwise(when($"provincia" === 16, lit("Pastaza"))
.otherwise(when($"provincia" === 17, lit("Pichincha"))
.otherwise(when($"provincia" === 18, lit("Tungurahua"))
.otherwise(when($"provincia" === 19, lit("Zamora Chinchipe"))
.otherwise(when($"provincia" === 20, lit("Galapagos"))
.otherwise(when($"provincia" === 21, lit("Sucumbios"))
.otherwise(when($"provincia" === 22, lit("Orellana"))
.otherwise(when($"provincia" === 23, lit("Santo Domingo de los Tsachilas"))
.otherwise(when($"provincia" === 24, lit("Santa  Elena"))
)   ))  )))))))))))))))))))))

In [7]:
val data3 = provincias.select($"provincia",$"genero")
val data4=data3.select($"genero",$"provincia").groupBy("genero","provincia").count()
val mujeres = data4.filter(data4("genero")=== "2 - Mujer").withColumnRenamed("count","Mujeres").select($"provincia",$"Mujeres")
val hombres = data4.filter(data4("genero")=== "1 - Hombre").withColumnRenamed("count","Hombres")select($"provincia",$"Hombres")
val resultado= hombres.as("hombres").join(mujeres.as("mujeres"), hombres("provincia") === mujeres("provincia")).select("hombres.provincia", "hombres", "mujeres")
resultado.show(false)

In [8]:
val  sueldos = provincias
.select("provincia","ingreso_laboral","grupo_ocupacion","nivel_de_instruccion")
.withColumnRenamed("nivel_de_instruccion","educacion")
.withColumnRenamed("grupo_ocupacion","ocupacion").orderBy(desc("ingreso_laboral")).limit(10)
sueldos.show(false)


sueldos.createOrReplaceTempView("tablFinal")

In [9]:
%%sql
select  * from tablFinal


In [10]:
datosDF.select("grupo_ocupacion").where($"edad" < 30).groupBy("grupo_ocupacion").count.orderBy(asc("grupo_ocupacion")).show

In [11]:
datosDF.select("nivel_de_instruccion").filter( "nivel_de_instruccion == '09 - Superior Universitario' ").count

In [12]:
datosDF.filter("grupo_ocupacion == '02 - Profesionales científicos e intelectuales'").filter("rama_actividad == '16 - P. Enseñanza'").select(avg("ingreso_laboral").as("sueldo promedio")).show

In [13]:
val desempleo = data2.select($"anio",$"mes",$"ingreso_laboral").withColumnRenamed("ingreso_laboral","salario")
val year15  = desempleo.filter(desempleo("anio") === 2015).filter("salario is null").count()
val year16 = desempleo.filter(desempleo("anio") === 2016).filter("salario is null").count()
val year17 = desempleo.filter(desempleo("anio") === 2017).filter("salario is null").count()
val year18 = desempleo.filter(desempleo("anio") === 2018).filter("salario is null").count()
val year19 = desempleo.filter(desempleo("anio") === 2019).filter("salario is null").count()

#### Para responder la pregunta anterior asumimos que los datos nulos son para personas que no trabajan.
#### Resultado obtenemos que el año con más desempleo es el 2019 es el año con, más desempleo

In [15]:
val AIL = data2.withColumn("range", when($"ingreso_laboral" >  0 and $"ingreso_laboral" <= 400, lit("1-400 Ingreso menor al salario básico"))
                .otherwise(when($"ingreso_laboral" > 400 and $"ingreso_laboral" <= 1000, lit("Ingreso entre  $400 y $1000"))
                .otherwise(when($"ingreso_laboral" > 1000 and $"ingreso_laboral" <= 2000, lit("Ingreso mayor a $2000"))
                )  ) ).groupBy("range").count().orderBy("range")
val  registros =data2.count.toDouble  
val tableAIL = AIL.select($"range",$"count"/registros*100).withColumnRenamed( "((count / 622776.0) * 100)","porcentaje")
tableAIL.show(false)
tableAIL.createOrReplaceTempView("AIL")


In [16]:
%%sql 
select * from AIL

In [17]:
val humanos = provincias.select($"provincia",$"anio",$"ingreso_laboral").filter(provincias("anio") === 2015 && provincias("ingreso_laboral") < 400).groupBy("provincia").count
//val registros  = humanos.select(sum("count"))
//registros.show
//val registros = humanos.s.count.toDouble 
val result = humanos.select($"provincia",$"count"/62130.0*100).withColumnRenamed("((count / 62130.0) * 100)","Porcentaje").withColumn("Porcentaje", format_number($"Porcentaje", 3))
result.show
result.select(sum("Porcentaje")).show

In [18]:
%%sql
